In [1]:
import sys
sys.path.append("..")
import torch
import torch.nn.functional as F
from omegaconf import OmegaConf
from ccreaim.model import operate, ae
from ccreaim.utils import dataset, cfg_classes, audio_tools, util

In [2]:
data_tar = "/scratch/other/sopi/CCREAIM/datasets/maestro_bank_data.tar"

load_trf_path = ""

context_length = 8

In [ ]:
# Load transformer 

trf_checkpoint = torch.load(load_trf_path, map_location="cpu")
trf_state_dict = trf_checkpoint["model_state_dict"]
hyper_cfg_schema = OmegaConf.structured(cfg_classes.HyperConfig)
trf_conf = OmegaConf.create(trf_checkpoint["hyper_config"])
trf_hyper_cfg = OmegaConf.merge(hyper_cfg_schema, trf_conf)
get_trf = operate.get_model_init_function(trf_hyper_cfg)
trf = get_trf()
trf.load_state_dict(trf_state_dict)
trf = trf.to(device)

In [ ]:
# Prepare datasets
tmp_data_root = dataset.prepare_dataset_on_tmp(data_tar)
data = dataset.BankTransformerDataset(tmp_data_root)
bank = dataset.Bank(tmp_data_root, context_length)